<a href="https://colab.research.google.com/github/joaoSouza2121/pln/blob/main/BOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Machine Learning

##<font color="blue">Processamento de Linguagem Natural</font>

# Modelo de Classificação de Idiomas de Sentenças com Bag of Words e PyTorch

In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torch import nn, optim

# Preparando os Dados

In [2]:
# Dados de treino
dados_treino = [("Tenho vinte paginas de leitura".lower().split(), "Portuguese"),
                ("I will visit the library".lower().split(), "English"),
                ("I am reading a book".lower().split(), "English"),
                ("This is my favourite chapter".lower().split(), "English"),
                ("Estou na biblioteca lendo meu livro preferido".lower().split(), "Portuguese"),
                ("Gosto de livros sobre viagens".lower().split(), "Portuguese")]


In [3]:
dados_treino

[(['tenho', 'vinte', 'paginas', 'de', 'leitura'], 'Portuguese'),
 (['i', 'will', 'visit', 'the', 'library'], 'English'),
 (['i', 'am', 'reading', 'a', 'book'], 'English'),
 (['this', 'is', 'my', 'favourite', 'chapter'], 'English'),
 (['estou', 'na', 'biblioteca', 'lendo', 'meu', 'livro', 'preferido'],
  'Portuguese'),
 (['gosto', 'de', 'livros', 'sobre', 'viagens'], 'Portuguese')]

In [4]:
# Dados de teste
dados_teste = [("Estou lendo".lower().split(), "Portuguese"),
               ("This is not my favourite book".lower().split(), "English")]


In [5]:
# Prepara o dicionário do vocabulário

# Dicionário para o vocabulário
dict_vocab = {}

# Contadoor
i = 0

# Loop pelos dados de treino e teste
for palavras, idiomas in dados_treino + dados_teste:
    for palavra in palavras:
        if palavra not in dict_vocab:
            dict_vocab[palavra] = i
            i += 1

In [6]:
# Visualiza o dicionario
print(dict_vocab)

{'tenho': 0, 'vinte': 1, 'paginas': 2, 'de': 3, 'leitura': 4, 'i': 5, 'will': 6, 'visit': 7, 'the': 8, 'library': 9, 'am': 10, 'reading': 11, 'a': 12, 'book': 13, 'this': 14, 'is': 15, 'my': 16, 'favourite': 17, 'chapter': 18, 'estou': 19, 'na': 20, 'biblioteca': 21, 'lendo': 22, 'meu': 23, 'livro': 24, 'preferido': 25, 'gosto': 26, 'livros': 27, 'sobre': 28, 'viagens': 29, 'not': 30}


In [7]:
# tamanho do corpus
tamanho_corpus = len(dict_vocab)

In [8]:
tamanho_corpus

31

In [9]:
# Numero de idiomas
idiomas = 2

In [10]:
# indice para os idiomas
label_index = {"Portuguese": 0, "English": 1}

# Construção do Modelo

In [16]:
# Classe para o modelo BOW de classificação
class ModeloBOW(nn.Module):  

    # Método construtor
    def __init__(self, lista_idiomas, tamanho_do_corpus):
        super(ModeloBOW, self).__init__()
        self.linear = nn.Linear(tamanho_do_corpus, lista_idiomas)
    # Feed Forward
    def forward(self, bow_vec):
        return F.log_softmax(self.linear(bow_vec), dim = 1)    

In [17]:
# Função para criar o vetor BOW necessário para o treinamento
def cria_bow_vetor(sentence, word_index):
    word_vec = torch.zeros(tamanho_corpus)
    for word in sentence:
        word_vec[dict_vocab[word]] += 1
    return word_vec.view(1, -1)


In [18]:
# Função para criar a variável target
def cria_target(label, label_index):
    return torch.LongTensor([label_index[label]])


In [19]:
# Cria o Modelo
modelo = ModeloBOW(idiomas, tamanho_corpus)

In [23]:
# Função de perda (loss)
loss_function = nn.NLLLoss()

In [21]:
# Optmizer
optimizer = optim.SGD(modelo.parameters(), lr= 0.1)

# Treinamento do Modelo

In [24]:
# Loop de treinamentoo
for epoch in range(100):
   
    for sentence, label in dados_treino:

        modelo.zero_grad()

        bow_vec = cria_bow_vetor(sentence, dict_vocab)
        target = cria_target(label, label_index)

        log_probs = modelo(bow_vec)

        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()
       
    if epoch % 10 == 0:
        print('Epoch: ', str(epoch+1),', Loss: ' + str(loss.item()))


Epoch:  1 , Loss: 0.997012197971344
Epoch:  11 , Loss: 0.09073120355606079
Epoch:  21 , Loss: 0.0449775867164135
Epoch:  31 , Loss: 0.02974196895956993
Epoch:  41 , Loss: 0.02217905968427658
Epoch:  51 , Loss: 0.01766929402947426
Epoch:  61 , Loss: 0.014677757397294044
Epoch:  71 , Loss: 0.012549597769975662
Epoch:  81 , Loss: 0.010958606377243996
Epoch:  91 , Loss: 0.009724742732942104


# Previsões e Avalições do Modelo

In [25]:
# Função para previsões
def faz_previsao(data):

    with torch.no_grad():
        sentence = data[0]
        label = data[1]
        bow_vec = cria_bow_vetor(sentence, dict_vocab)
        log_probs = modelo(bow_vec)
        print(sentence)
        print('Probabilidade de ser o label: ' + label, 'é igual a: ',  np.exp(log_probs))


In [26]:
# Previsão com a primeira sentença de teste
faz_previsao(dados_teste[0])

['estou', 'lendo']
Probabilidade de ser o label: Portuguese é igual a:  tensor([[0.7967, 0.2033]])


In [27]:
dados_teste[0]

(['estou', 'lendo'], 'Portuguese')

In [28]:
# Previsão da segunda sentença
faz_previsao(dados_teste[1])

['this', 'is', 'not', 'my', 'favourite', 'book']
Probabilidade de ser o label: English é igual a:  tensor([[0.0088, 0.9912]])


##Previsões com Novas Frases

In [29]:
# Nova frase
novas_frases = [("Tenho livros sobre viagens".lower().split(), "Portuguese"),
                ("Estou escrevendo".lower().split(), "Portuguese"),
                ("Gosto de biblioteca".lower().split(), "Portuguese")]


In [30]:
novas_frases

[(['tenho', 'livros', 'sobre', 'viagens'], 'Portuguese'),
 (['estou', 'escrevendo'], 'Portuguese'),
 (['gosto', 'de', 'biblioteca'], 'Portuguese')]

In [31]:
faz_previsao(novas_frases[0])

['tenho', 'livros', 'sobre', 'viagens']
Probabilidade de ser o label: Portuguese é igual a:  tensor([[0.9714, 0.0286]])


In [32]:
faz_previsao(novas_frases[2])

['gosto', 'de', 'biblioteca']
Probabilidade de ser o label: Portuguese é igual a:  tensor([[0.9582, 0.0418]])
